In [1]:
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 토크나이저와 모델
model = AutoModelForTokenClassification.from_pretrained('./koelectra_v3_ner_model5')
tokenizer = AutoTokenizer.from_pretrained('./koelectra_v3_ner_model5')
file_path = './data/test/test_data+text.json'

In [3]:
# data 불러오기
def load_data(file_path):
    """JSON 파일에서 NER 데이터 로드"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)
        

In [4]:
# KoNERDataset전용 클래스 생성
class KoNERDataset(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [ ]:
def prepare_koelectra_tagged_data(data, tokenizer, max_length=64):
    """KoElectra용 데이터 준비"""
    text = [d['text'] for d in data]
    tokens = [d['tokens'] for d in data]
    labels = [d['ner_tags'] for d in data]

    # KoElectra 토크나이저로 인코딩
    encodings = tokenizer(
        tokens, 
        is_split_into_words=True, 
        padding=True, 
        truncation=True, 
        max_length=max_length,
        return_tensors='pt'
    )

    # # NER 라벨 처리
    # new_labels = []
    # for i, label in enumerate(labels):
    #     word_ids = encodings.word_ids(batch_index=i)
    #     label_ids = [-100] * len(word_ids)  # 초기값 -100으로 설정
        
    #     # 개선된 라벨 매핑 로직
    #     for word_idx, tag in enumerate(label):
    #         # word_ids에서 해당 word_idx의 첫 번째 인덱스 찾기
    #         indices = [j for j, w_id in enumerate(word_ids) if w_id == word_idx]
            
    #         if indices:
    #             # 첫 번째 인덱스에 태그 할당
    #             label_ids[indices[0]] = tag
        
    #     new_labels.append(label_ids)

    return text, encodings, labels

In [6]:
def test_koelectra_ner_model(model, test_dataset):
    """모델 테스트"""

    model.to(device)
    model.eval()

    all_preds = [] 
    all_labels = []
    all_tokens = [] 

    with torch.no_grad():       
        for batch in test_dataset:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']
                        
            outputs = model(
                input_ids, 
                attention_mask=attention_mask)
            
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            # 예측과 레이블만 수집
            for pred, label, input_id in zip(predictions, labels, input_ids):
                mask = label != -100
                all_preds.extend(pred[mask].cpu().numpy())
                all_labels.extend(label[mask].numpy())
                print(f"all_preds {all_preds}")
                print(f"all_labels {all_labels}")

                # 원본 토큰으로 복원
                tokens = tokenizer.convert_ids_to_tokens(input_id)
                all_tokens.extend([token for token, m in zip(tokens, mask) if m])       
                print(f"all_tokens {all_tokens}")

    # 상세 분류 보고서 출력
    print("Classification Report_[test]:")
    print(classification_report(all_labels, all_preds))

    # 일부 잘못 예측된 케이스 출력 (선택사항)
    print("\n일부 잘못 예측된 예시:")
    for token, true_label, pred_label in zip(all_tokens, all_labels, all_preds):
        if true_label != pred_label:
            print(f"Token: {token}, True Label: {true_label}, Predicted Label: {pred_label}")

In [ ]:
def main_test():

    # 테스트 데이터 로드 
    test_data = load_data(file_path)

    # 테스트 데이터 준비 및 데이터셋 생성
    test_text, test_encodings, test_labels = prepare_koelectra_tagged_data(test_data, tokenizer)

    # 테스트 데이터셋 생성
    test_dataset = DataLoader(
        KoNERDataset(test_encodings, test_labels), 
        batch_size=8, 
        shuffle=False
    )

    # 모델 테스트
    test_koelectra_ner_model(model, test_dataset)

if __name__ == '__main__':
    main_test()

79
79


C:\Users\KDP-23\AppData\Local\Temp\ipykernel_6764\4259181314.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


RuntimeError: stack expects each tensor to be equal size, but got [35] at entry 0 and [34] at entry 1